employee1.csv → HR source (employee_id)

employee2.csv → Payroll / system source (id)

Goal: match employee_id ↔ id for the same persons

Matching columns:

name

supervisor_name

business_title (needs cleanup)

job_start_date
location

git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https:
git push -u origin main

employee1.csv → HR source (employee_id)

employee2.csv → Payroll / system source (id)

Goal: match employee_id ↔ id for the same persons

Matching columns:

name

supervisor_name

business_title (needs cleanup)

job_start_date
location

git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin https://github.com/dhailu/fuzzy_matches.git
git push -u origin main

In [1]:
import pandas as pd
from rapidfuzz import fuzz

#### Load CSV files into pandas

In [ ]:
# Create a sample DataFrame -- title_clean
emp1_csv = 'employeeCC.csv'
emp2_csv = 'employeeAZ.csv'
emp1 = pd.read_csv(emp1_csv)
emp2 = pd.read_csv(emp2_csv)
display(emp1)
display(emp2)

,employee_id,first_name,last_name,middle_name,full_name,supervisor_id,Supervisor_name,start_date,business_title,location
0,11112,Smith,Tom,M,"Tom,Smith M",22223,"John, Smith W",1/1/2024,technician,RG1
1,11113,Teshale,Alemu,R,"Alemu,Teshale",22227,"Alex, Raja",1/1/2024,engineer,TTR
2,11114,Teresa,Jonson,NaN,"Jonson,Teresa",22228,"John, Smith W",1/2/2024,technician,TTR
3,11115,Abel,Gery,NaN,"Gery,Abel",22223,"Raji,Jerald",1/3/2024,HR,RG1
4,11116,Beza,Tesema,NaN,"Tesema,Beza R",22229,"John, Smith W",1/4/2024,Finance,RG1


,id,employee_name,Supv_name,job_start_date,business_title,location
0,22223,"Tom,Smith","John, Smith",1/1/2024,technician- DDR,RG1
1,22224,"Alemu,Teshale","Alex, Raja",1/2/2024,Sr. engineer RRT,TTR
2,22225,"Jonson,Teresa","John, Smith",1/3/2024,technician (TTR),TTR
3,22226,"Gerra,Abel","Raji,Jerald",1/4/2024,HR (TTR),RG1
4,22227,"Tesema,Beza","John, Smith W",1/4/2024,Finance,RG1


#### Normalize & clean data (VERY IMPORTANT)
Standardize text

In [3]:
def normalize_text(x):
    if pd.isna(x):
        return ""
    return (
        str(x)
        .lower()
        .replace(",", " ")
        .replace("-", " ")
        .strip()
    )

#### Apply to relevant columns:

In [4]:
cols_emp1 = ["full_name", "Supervisor_name", "business_title", "location"]
cols_emp2 = ["employee_name", "Supv_name", "business_title", "location"]

for c in cols_emp1:
    emp1[c] = emp1[c].apply(normalize_text)

for c in cols_emp2:
    emp2[c] = emp2[c].apply(normalize_text)

#### Clean business title (your key requirement)

Example:

technician

technician - rrt

technician (ttr)
##### Normalize job titles

In [ ]:
# def clean_title(title):
#     title = normalize_text(title)
#     return title.split()[0]   # keep main role only



import re
import pandas as pd

import re
import pandas as pd

def clean_table2_title_suffix(title):
    if pd.isna(title):
        return ""

    title = title.strip()

    # Remove trailing parentheses suffix: (CCB), (brt), (TEMP)
    title = re.sub(r"\s*\([^)]*\)\s*$", "", title)

    # Remove trailing dash suffix: -CC, -RRT, -TEMP
    title = re.sub(r"\s*-\s*[A-Za-z0-9]+\s*$", "", title)

    return title.strip()

emp2["clean_title"] = emp2["business_title"].apply(clean_table2_title_suffix)

#### Prepare date fields

Dates should match exactly, not fuzzily.

In [11]:
emp1["start_date"] = pd.to_datetime(emp1["start_date"])
emp2["job_start_date"] = pd.to_datetime(emp2["job_start_date"])

#### Fuzzy matching logic (Levenshtein)

We’ll use a weighted score across columns.

In [12]:
from rapidfuzz import fuzz

#### Composite similarity fuzzy

In [17]:
def match_score(r1, r2):
    name_score = fuzz.token_sort_ratio(r1["full_name"], r2["employee_name"])
    supv_score = fuzz.token_sort_ratio(r1["Supervisor_name"], r2["Supv_name"])
    title_score = fuzz.ratio(r1["business_title"], r2["business_title"])
    loc_score = fuzz.ratio(r1["location"], r2["location"])

    date_score = 100 if r1["start_date"] == r2["job_start_date"] else 0

    final_score = (
        0.40 * name_score +
        0.20 * supv_score +
        0.15 * title_score +
        0.15 * loc_score +
        0.10 * date_score
    )

    return round(final_score, 2)

#### Perform matching

 This is O(n²) — OK for CSV demos.
(We’ll optimize later for big data.)

In [18]:
matches = []

for _, r1 in emp1.iterrows():
    best_match = None
    best_score = 0

    for _, r2 in emp2.iterrows():
        score = match_score(r1, r2)

        if score > best_score:
            best_score = score
            best_match = r2

    if best_score >= 80:  # confidence threshold
        matches.append({
            "employee_id": r1["employee_id"],
            "matched_id": best_match["id"],
            "full_name": r1["full_name"],
            "matched_name": best_match["employee_name"],
            "Supervisor_name": r1["Supervisor_name"],
            "matched_Supv_name": best_match["Supv_name"],
            "business_title": r1["business_title"],
            "matched_business_title": best_match["business_title"],
            "location": r1["location"],
            "matched_location": best_match["location"],
            "start_date": r1["start_date"],
            "matched_job_start_date": best_match["job_start_date"],
            
            "score": best_score
        })

#### Final matched dataframe

In [20]:
result_df = pd.DataFrame(matches)
print(result_df)

   employee_id  matched_id      full_name   matched_name Supervisor_name  \
0        11112       22223    tom smith m      tom smith   john  smith w   
1        11113       22224  alemu teshale  alemu teshale      alex  raja   
2        11114       22225  jonson teresa  jonson teresa   john  smith w   
3        11116       22227  tesema beza r    tesema beza   john  smith w   

  matched_Supv_name business_title matched_business_title location  \
0       john  smith     technician       technician   ddr      rg1   
1        alex  raja       engineer      sr. engineer  rrt      ttr   
2       john  smith     technician       technician (ttr)      ttr   
3     john  smith w        finance                finance      rg1   

  matched_location start_date matched_job_start_date  score  
0              rg1 2024-01-01             2024-01-01  90.72  
1              ttr 2024-01-01             2024-01-02  84.60  
2              ttr 2024-01-02             2024-01-03  84.72  
3              rg1 2